In [1]:
#importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

Loading the Image and Extracting Light Coordinates:

- The image is loaded
- The coordinates of the illuminated pixels (above a certain threshold) are extracted.


In [2]:
def load_image(image_address):
    # Load the image and extract coordinates of lights
    image = Image.open(image_address).convert('L')
    
    image_array = np.array(image)
    threshold = 200
    light_coordinates = np.argwhere(image_array > threshold)
    return light_coordinates

Now we have to create a function to apply K-means clustering which:
- Randomly initialize centroids.
- Iteratively assign points to the nearest centroid and update centroids until convergence or the maximum number of iterations (set to 100) is reached.

In [3]:
# Implementing K-Means Clustering
def k_means(points, k, max_iters=100):
    np.random.seed(0)
    centroids = points[np.random.choice(points.shape[0], k, replace=False)]
    for _ in range(max_iters):
        distances = np.linalg.norm(points[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centroids = np.array([points[labels == i].mean(axis=0) for i in range(k)])
        if np.all(centroids == new_centroids):
             break
        centroids = new_centroids
    return labels, centroids

Now let us create a function to compute the sum of squared distances between points and their assigned centroids.

In [4]:
def calculate_ssd(points, labels, centroids):
    ssd = 0
    for i in range(len(centroids)):
        cluster_points = points[labels == i]
        ssd += np.sum((cluster_points - centroids[i]) ** 2)
    return ssd

Now create a list which stores the SSDs corresponding to values of K from 1 to 10.

In [5]:
def create_ssds_list(light_coordinates):
    ssds = []
    K = range(1, 11)
    for k in K:
        labels, centroids = k_means(light_coordinates, k)
        ssd = calculate_ssd(light_coordinates, labels, centroids)
        ssds.append(ssd)
    return ssds

Now let us create a function to plot SSDs vs k (so that we can visualise the elbow point) 

In [6]:
# Plot the SSDs for different values of k
def plot(K,ssds):
    plt.plot(K, ssds, 'bx-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Sum of squared distances (SSD)')
    plt.title(f'Elbow Method For Optimal k')
    plt.show()


Now to calculate the value of k at which elbow formation occurs, we will find the point in the plot which is farthest from the line joining the first and last point. For this task, we will create a function find_elbow_point.

In [7]:
# Automatic detection of the elbow point
def find_elbow_point(ssds):
    n_points = len(ssds)
    k_values = [i for i in range(n_points)]
    ssd_values = ssds
    
    all_coords = np.zeros((n_points, 2))
    for i in range(n_points):
        all_coords[i, 0] = k_values[i]
        all_coords[i, 1] = ssd_values[i]
        
    first_point = all_coords[0]
    last_point = all_coords[-1]
    line_vec = last_point - first_point
    line_vec_norm = line_vec / np.linalg.norm(line_vec)
    vec_from_first = all_coords - first_point
    scalar_product = np.dot(vec_from_first, line_vec_norm)
    vec_from_first_parallel = np.outer(scalar_product, line_vec_norm)
    vec_to_line = vec_from_first - vec_from_first_parallel
    distances_to_line = np.linalg.norm(vec_to_line, axis=1)
    elbow_index = np.argmax(distances_to_line)
    return elbow_index+1;

To display the clustered cities in different colours, we will use the matplotlib library.

In [8]:
def plot_clusters(points, labels, centroids, k):
    plt.figure(figsize=(6,6))
    colors = plt.colormaps['tab10'](np.linspace(0, 1, k))
    for i in range(k):
        cluster_points = points[labels == i]
        plt.scatter(cluster_points[:, 1], cluster_points[:, 0], s=10, color=colors[i], label=f'City_{i+1}')
    plt.scatter(centroids[:, 1], centroids[:, 0], s=80, c='black', marker='X', label='Centroids')
    plt.legend()
    plt.gca().invert_yaxis()
    plt.title(f'Clustered Cities from Night-Time Satellite Image')
    plt.show()
    


We have got the coordinates of the city centers. Now we have to calculate the distance between the centers. We will do this by creating a function and passing the centroids array in it.

In [9]:
# Calculate the distances between the centers
def calculate_pairwise_distances(centroids):
    l = len(centroids)
    distance_matrix = np.zeros((l, l))
    for i in range(l):
        for j in range(i + 1, l):
            distance = np.linalg.norm(centroids[i] - centroids[j])
            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance
    return distance_matrix
   


Finally we can print this array by creating a data frame.

In [10]:
def table(centroids,optimal_k):
    distance_matrix_square = calculate_pairwise_distances(centroids)
    
    # Presenting the distances in a tabular format
    distance_df = pd.DataFrame(distance_matrix_square, 
                               index=[f'City_{i+1}' for i in range(optimal_k)], 
                               columns=[f'City_{i+1}' for i in range(optimal_k)])
    
    print(distance_df)

Now we can create a final function to process a image using all the functions created above.

In [11]:
def process_image(image_address):

    light_coordinates=load_image(image_address)
    
    ssds=create_ssds_list(light_coordinates)
    K = range(1, 11)

    plot(K,ssds)

    optimal_k = find_elbow_point(ssds)
    print(f'Optimal number of clusters (k): {optimal_k}')
    
    #Run k-means again with the optimal k
    labels, centroids = k_means(light_coordinates, optimal_k)

    #plot clustered cities
    plot_clusters(light_coordinates, labels, centroids, optimal_k)

    print(f'Distances between city centers:')
    table(centroids,optimal_k)

Now we can process our images one by one.

In [ ]:
process_image('img.png')

FileNotFoundError: [Errno 2] No such file or directory: '1.png'